### Ensure Project Root Is Available

In [41]:
import sys
import os

# Walk upward until we find 'src'
current = os.getcwd()

while current != os.path.dirname(current):
    if "src" in os.listdir(current):
        if current not in sys.path:
            sys.path.append(current)
        print("Project root added:", current)
        break
    current = os.path.dirname(current)

Project root added: d:\Python\Vidhi


### Imports

In [42]:
from uuid import uuid4
from pprint import pprint
import traceback

from src.orchestrator.agent_orchestrator import AgentOrchestrator
from src.agents.lii_issue_identifier_agent import LIIIssueIdentifierAgent

### Initialize Orchestrator

In [43]:
issue_spotter = LIIIssueIdentifierAgent()
orchestrator = AgentOrchestrator(agents=[issue_spotter])
orchestrator.issue_spotter_agent = issue_spotter

session_id = str(uuid4())

print("Session ID:", session_id)

Session ID: 7ecafaaa-7540-4d2a-8365-a022eb188799


### Scenario 1 - Empty Input

In [44]:
try:
    result = orchestrator.run({
        "case_description": "",
        "jurisdiction": "Maharashtra High Court",
        "document_type": "Legal Opinion",
    })
    pprint(result)
except Exception as e:
    print("❌ Empty Input Failure Triggered")
    traceback.print_exc()

2026-02-15 19:21:27,874 - src.orchestrator.agent_orchestrator - ERROR - Request validation failed: Missing required field: user_query


{'agent_results': [],
 'citation_validation': None,
 'error': 'Missing required field: user_query',
 'execution_time_sec': 0.0031,
 'final_answer': '',
 'hallucination_validation': None,
 'message': 'Invalid request payload.',
 'output': {},
 'success': False}


### Scenario 2 - Garbage / Malformed Input

In [45]:
try:
    result = orchestrator.run({
        "case_description": "@@@@ ### 12345 random text with no legal meaning $$$$",
        "jurisdiction": "Maharashtra High Court",
        "document_type": "Legal Opinion",
    })
    pprint(result)
except Exception as e:
    print("❌ Malformed Input Failure Triggered")
    traceback.print_exc()

2026-02-15 19:21:27,896 - src.orchestrator.agent_orchestrator - INFO - Orchestration started. request_id=5db2d0cc-2ce3-4d87-81ca-b36bcda332ab
2026-02-15 19:21:27,898 - src.orchestrator.agent_orchestrator - INFO - Executing agent: LIIIssueIdentifierAgent
2026-02-15 19:21:27,899 - src.orchestrator.agent_orchestrator - INFO - Agent LIIIssueIdentifierAgent executed successfully in 0.001s
2026-02-15 19:21:27,899 - src.orchestrator.agent_orchestrator - INFO - Orchestration finished. request_id=5db2d0cc-2ce3-4d87-81ca-b36bcda332ab, time=0.0029s


{'agent_results': [{'agent_name': 'LIIIssueIdentifierAgent',
                    'error': None,
                    'execution_time_sec': 0.001,
                    'output': {'agent': 'LIIIssueIdentifierAgent',
                               'data': {'primary_issues': ['General '
                                                           'Civil/Legal '
                                                           'Dispute (requires '
                                                           'detailed '
                                                           'classification)'],
                                        'secondary_issues': [],
                                        'summary': 'Identified 1 primary '
                                                   'issue(s). Jurisdiction '
                                                   'considered: Maharashtra '
                                                   'High Court. Document type: '
                                       

### Scenario 3 - Contradictory Timeline

In [46]:
contradictory_case = """
Tenant was evicted in March 2023.
However, eviction notice was issued in June 2024.
Rent agreement expired in 2025.
"""

try:
    result = orchestrator.run({
        "case_description": contradictory_case,
        "jurisdiction": "Maharashtra High Court",
        "document_type": "Legal Opinion",
    })
    pprint(result)
except Exception as e:
    print("❌ Logical Timeline Failure Triggered")
    traceback.print_exc()

2026-02-15 19:21:27,925 - src.orchestrator.agent_orchestrator - INFO - Orchestration started. request_id=ef49b3d9-7a92-4462-8bf6-bd6ecf7f771a
2026-02-15 19:21:27,926 - src.orchestrator.agent_orchestrator - INFO - Executing agent: LIIIssueIdentifierAgent
2026-02-15 19:21:27,927 - src.orchestrator.agent_orchestrator - INFO - Agent LIIIssueIdentifierAgent executed successfully in 0.0012s
2026-02-15 19:21:27,929 - src.orchestrator.agent_orchestrator - INFO - Orchestration finished. request_id=ef49b3d9-7a92-4462-8bf6-bd6ecf7f771a, time=0.0048s


{'agent_results': [{'agent_name': 'LIIIssueIdentifierAgent',
                    'error': None,
                    'execution_time_sec': 0.0012,
                    'output': {'agent': 'LIIIssueIdentifierAgent',
                               'data': {'primary_issues': ['Breach of Contract '
                                                           '/ Contractual '
                                                           'Dispute',
                                                           'Tenancy / Eviction '
                                                           'Dispute'],
                                        'secondary_issues': ['Notice / Legal '
                                                             'Notice '
                                                             'Requirements'],
                                        'summary': 'Identified 2 primary '
                                                   'issue(s) and 1 secondary '
                          

### Scenario 4 - Oversized Input

In [47]:
large_input = "Breach of contract. " * 5000  # artificially large

try:
    result = orchestrator.run({
        "case_description": large_input,
        "jurisdiction": "Maharashtra High Court",
        "document_type": "Legal Opinion",
    })
    pprint(result)
except Exception as e:
    print("❌ Large Input Failure Triggered")
    traceback.print_exc()

2026-02-15 19:21:27,949 - src.orchestrator.agent_orchestrator - INFO - Orchestration started. request_id=e2a54cf4-8b3b-4f0b-9a24-44671e50b6f3
2026-02-15 19:21:27,950 - src.orchestrator.agent_orchestrator - INFO - Executing agent: LIIIssueIdentifierAgent
2026-02-15 19:21:27,954 - src.orchestrator.agent_orchestrator - INFO - Agent LIIIssueIdentifierAgent executed successfully in 0.0042s
2026-02-15 19:21:27,956 - src.orchestrator.agent_orchestrator - INFO - Orchestration finished. request_id=e2a54cf4-8b3b-4f0b-9a24-44671e50b6f3, time=0.007s


{'agent_results': [{'agent_name': 'LIIIssueIdentifierAgent',
                    'error': None,
                    'execution_time_sec': 0.0042,
                    'output': {'agent': 'LIIIssueIdentifierAgent',
                               'data': {'primary_issues': ['Breach of Contract '
                                                           '/ Contractual '
                                                           'Dispute'],
                                        'secondary_issues': [],
                                        'summary': 'Identified 1 primary '
                                                   'issue(s). Jurisdiction '
                                                   'considered: Maharashtra '
                                                   'High Court. Document type: '
                                                   'Legal Opinion.'},
                               'errors': [],
                               'metadata': {'agent': 'LIIIssueIdentif

### Inject Agent Failure

In [48]:
# Monkey patch an agent method (temporary for testing)

original_method = orchestrator.issue_spotter_agent.run

def failing_agent(*args, **kwargs):
    raise RuntimeError("Simulated agent failure")

orchestrator.issue_spotter_agent.run = failing_agent

try:
    result = orchestrator.run({
        "case_description": "Tenant eviction dispute due to non-payment of rent.",
        "jurisdiction": "Maharashtra High Court",
        "document_type": "Legal Opinion",
    })
    pprint(result)
except Exception as e:
    print("❌ Internal Agent Failure Triggered")
    traceback.print_exc()

# Restore original behavior
orchestrator.issue_spotter_agent.run = original_method

2026-02-15 19:21:27,975 - src.orchestrator.agent_orchestrator - INFO - Orchestration started. request_id=176da9e4-2577-482c-9f8e-84a2032f4f34
2026-02-15 19:21:27,976 - src.orchestrator.agent_orchestrator - INFO - Executing agent: LIIIssueIdentifierAgent
2026-02-15 19:21:27,977 - src.orchestrator.agent_orchestrator - ERROR - Agent execution failed: LIIIssueIdentifierAgent error=Agent 'LIIIssueIdentifierAgent' failed during execution: Simulated agent failure
2026-02-15 19:21:27,978 - src.orchestrator.agent_orchestrator - WARNING - Pipeline completed but produced no output.


{'agent_results': [{'agent_name': 'LIIIssueIdentifierAgent',
                    'error': "Agent 'LIIIssueIdentifierAgent' failed during "
                             'execution: Simulated agent failure',
                    'execution_time_sec': 0.001,
                    'output': {},
                    'success': False}],
 'citation_validation': None,
 'error': 'Pipeline returned empty output.',
 'execution_time_sec': 0.0035,
 'final_answer': '',
 'hallucination_validation': None,
 'message': 'No output generated by agents.',
 'output': {},
 'success': False}
